<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_yolo_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR YOLOv8-SEGMENTATION OPTIMIZADO PARA FOTOGRAFÍA DE RETRATO
================================================================================
Sistema de evaluación para modelos YOLOv8-seg con estructura estandarizada
y optimizaciones específicas para fotografía de retrato.

CARACTERÍSTICAS PRINCIPALES:
- Procesamiento incremental con sistema de checkpoint robusto
- Gestión óptima de memoria para Google Colab gratuito
- Guardado automático después de cada imagen procesada
- Configuraciones optimizadas para fotografía de retrato
- Filtrado inteligente de personas basado en heurísticas del estado del arte
- Post-procesamiento avanzado: NMS, área, aspect ratio, posición
- Visualizaciones 3-panel avanzadas (Original + Detecciones + Máscaras)
- NPZ completo con todas las máscaras y scores para análisis posterior
- JSON ligero con métricas de rendimiento

OPTIMIZACIONES PARA RETRATO:
- Configuraciones IOU/CONF específicas para personas
- Filtrado por área: 5%-90% de la imagen (personas significativas)
- Filtrado por aspect ratio: 0.3-1.5 (personas son más altas que anchas)
- Priorización de detecciones centrales
- NMS agresivo para evitar duplicados en retratos
- 3 configuraciones predefinidas: fast, balanced, quality

ALMACENAMIENTO NPZ (por imagen y umbral):
- masks: Máscaras binarias (N, H, W) en uint8

Nota: Imagen original en /TFM/0_Imagenes/
      Geometría calculada en notebook 03 con Shapely/Mahotas
      Resto de información en JSON por imagen

VISUALIZACIONES 3-PANEL:
- Panel 1: Imagen original sin modificar
- Panel 2: Bounding boxes + IDs + scores de confianza
- Panel 3: Máscaras coloreadas con overlay semi-transparente

ESTRUCTURA DE SALIDA:
/TFM/2_Modelos/yolov8/evaluacion/
├── {modelo}_{config}/
│   ├── json/                         # UN JSON POR IMAGEN
│   ├── mascaras/                     # UN NPZ POR IMAGEN
│   ├── visualizaciones/              # UN PNG POR IMAGEN
│   └── checkpoint.json               # Estado del procesamiento

Entrada: Imágenes desde /TFM/0_Imagenes/
Salida: JSON, máscaras NPZ completas y visualizaciones en estructura organizada

Referencias:
- Jocher et al. (2023) "Ultralytics YOLOv8"
- Lin et al. (2014) "Microsoft COCO: Common Objects in Context"
- Dollar et al. (2012) "Pedestrian Detection: An Evaluation"

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña (UOC)
Fecha: Octubre 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR YOLOv8-SEGMENTATION OPTIMIZADO PARA FOTOGRAFÍA DE RETRATO\n================================================================================\nSistema de evaluación para modelos YOLOv8-seg con estructura estandarizada\ny optimizaciones específicas para fotografía de retrato.\n\nCARACTERÍSTICAS PRINCIPALES:\n- Procesamiento incremental con sistema de checkpoint robusto\n- Gestión óptima de memoria para Google Colab gratuito\n- Guardado automático después de cada imagen procesada\n- Configuraciones optimizadas para fotografía de retrato\n- Filtrado inteligente de personas basado en heurísticas del estado del arte\n- Post-procesamiento avanzado: NMS, área, aspect ratio, posición\n- Visualizaciones 3-panel avanzadas (Original + Detecciones + Máscaras)\n- NPZ completo con todas las máscaras y scores para análisis posterior\n- JSON ligero con métricas de rendimiento\n\nOPTIMIZACIONES PARA RETRATO:\n-

In [ ]:
# =============================================================================
# INSTALACIÓN DE DEPENDENCIAS (EJECUTAR EN COLAB)
# =============================================================================

print("="*80)
print("INSTALANDO DEPENDENCIAS PARA YOLOv8-SEGMENTATION")
print("="*80)

# Instalar ultralytics (incluye YOLOv8)
print("\n[1/3] Instalando Ultralytics YOLOv8...")
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])

# Dependencias adicionales
print("[2/3] Instalando librerías de procesamiento...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "opencv-python"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "Pillow"])

print("[3/3] Verificando PyTorch y otras dependencias...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "matplotlib", "numpy", "scipy"])

print("\n" + "="*80)
print("OK DEPENDENCIAS INSTALADAS CORRECTAMENTE")
print("="*80 + "\n")

# =============================================================================
# MONTAR GOOGLE DRIVE
# =============================================================================

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("OK Google Drive montado correctamente\n")
except:
    IN_COLAB = False
    print("ADVERTENCIA No estamos en Google Colab\n")

INSTALANDO DEPENDENCIAS PARA YOLOv8-SEGMENTATION

[1/3] Instalando Ultralytics YOLOv8...
[2/3] Instalando librerías de procesamiento...
[3/3] Verificando PyTorch y otras dependencias...

OK DEPENDENCIAS INSTALADAS CORRECTAMENTE

Mounted at /content/drive
OK Google Drive montado correctamente



In [ ]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import gc
import time
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from ultralytics import YOLO

warnings.filterwarnings('ignore')

print("="*80)
print("OK LIBRERÍAS IMPORTADAS CORRECTAMENTE")
print("="*80)

# Verificar disponibilidad de GPU
if torch.cuda.is_available():
    print(f"OK GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"  VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("ADVERTENCIA GPU no disponible - Se usará CPU (más lento)")

print("="*80 + "\n")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
OK LIBRERÍAS IMPORTADAS CORRECTAMENTE
OK GPU disponible: Tesla T4
  VRAM: 14.74 GB



In [ ]:
# =============================================================================
# CONFIGURACIÓN
# =============================================================================

@dataclass
class ModeloYOLO:
    """Información de un modelo YOLOv8-seg"""
    nombre: str
    nombre_corto: str
    archivo_pesos: str
    tamaño: str
    parametros: str

MODELOS_DISPONIBLES = {
    'nano': ModeloYOLO(
        nombre='YOLOv8 Nano Segmentation',
        nombre_corto='nano',
        archivo_pesos='yolov8n-seg.pt',
        tamaño='nano',
        parametros='3.4M'
    ),
    'small': ModeloYOLO(
        nombre='YOLOv8 Small Segmentation',
        nombre_corto='small',
        archivo_pesos='yolov8s-seg.pt',
        tamaño='small',
        parametros='11.8M'
    ),
    'medium': ModeloYOLO(
        nombre='YOLOv8 Medium Segmentation',
        nombre_corto='medium',
        archivo_pesos='yolov8m-seg.pt',
        tamaño='medium',
        parametros='27.3M'
    ),
    'large': ModeloYOLO(
        nombre='YOLOv8 Large Segmentation',
        nombre_corto='large',
        archivo_pesos='yolov8l-seg.pt',
        tamaño='large',
        parametros='46.0M'
    ),
    'xlarge': ModeloYOLO(
        nombre='YOLOv8 XLarge Segmentation',
        nombre_corto='xlarge',
        archivo_pesos='yolov8x-seg.pt',
        tamaño='xlarge',
        parametros='71.8M'
    )
}

In [ ]:
@dataclass
class ConfiguracionUmbrales:
    """
    Configuración de umbrales optimizada para fotografía de retrato.

    Referencias:
    - Ultralytics YOLOv8 documentation (2023)
    - COCO dataset best practices (Lin et al., 2014)

    Parámetros clave para retrato:
    - conf: Umbral de confianza para filtrar detecciones débiles
    - iou: Umbral NMS para evitar duplicados (crítico en retratos)
    - max_det: Número máximo de detecciones (limitado para retratos)
    """
    nombre: str
    valores_conf: List[float]
    iou_threshold: float
    max_det: int
    descripcion: str

    def __post_init__(self):
        """Validación de parámetros"""
        if not all(0 <= v <= 1 for v in self.valores_conf):
            raise ValueError("Los valores de confianza deben estar entre 0 y 1")
        if not 0 <= self.iou_threshold <= 1:
            raise ValueError("IOU threshold debe estar entre 0 y 1")

CONFIGURACIONES_UMBRALES = {
    'fast_portrait': ConfiguracionUmbrales(
        nombre='fast_portrait',
        valores_conf=[0.25, 0.4, 0.5],
        iou_threshold=0.6,  # NMS más agresivo para evitar duplicados
        max_det=10,          # Limitado: enfoque en personas principales
        descripcion='Rápida - Detecta personas principales con alta confianza'
    ),
    'balanced_portrait': ConfiguracionUmbrales(
        nombre='balanced_portrait',
        valores_conf=[0.15, 0.25, 0.35, 0.5],
        iou_threshold=0.5,
        max_det=15,
        descripcion='Equilibrada - Balance entre velocidad y detección de personas secundarias'
    ),
    'sensitive_portrait': ConfiguracionUmbrales(
        nombre='sensitive_portrait',
        valores_conf=[0.05, 0.1, 0.2, 0.3, 0.4],
        iou_threshold=0.45,  # Menos agresivo para capturar más personas
        max_det=20,
        descripcion='Sensible - Detecta incluso personas parcialmente visibles o en segundo plano'
    ),
    'quality_portrait': ConfiguracionUmbrales(
        nombre='quality_portrait',
        valores_conf=[0.1, 0.2, 0.3, 0.5, 0.7],
        iou_threshold=0.5,
        max_det=15,
        descripcion='Calidad - Rango amplio para análisis de sensibilidad en retratos'
    )
}

@dataclass
class ConfiguracionEvaluacion:
    """Configuración principal del sistema"""
    # Modelos a evaluar
    modelos_evaluar: List[str] = None  # None = todos

    # Configuraciones de umbrales a evaluar
    configs_umbrales: List[str] = None  # None = ['balanced_portrait']

    # Rutas
    ruta_base: Path = Path("/content/drive/MyDrive/TFM")
    ruta_imagenes: Path = None
    ruta_resultados: Path = None

    # Procesamiento
    max_dimension: int = 1024
    img_size: int = 640
    pausa_entre_imagenes: float = 2.0
    pausa_entre_modelos: float = 5.0

    def __post_init__(self):
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "2_Modelos" / "yolov8"
        if self.modelos_evaluar is None:
            self.modelos_evaluar = list(MODELOS_DISPONIBLES.keys())
        if self.configs_umbrales is None:
            # DEFAULT: balanced_portrait (recomendado para fotografía de retrato)
            self.configs_umbrales = ['balanced_portrait']

In [ ]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares"""

    @staticmethod
    def cargar_imagen(ruta: Path, max_dim: int = None) -> Tuple[Image.Image, Dict]:
        """Carga imagen con opción de redimensionar"""
        img = Image.open(ruta).convert("RGB")
        dim_orig = img.size

        redim = False
        if max_dim and max(img.size) > max_dim:
            ratio = max_dim / max(img.size)
            new_size = tuple(int(d * ratio) for d in img.size)
            img = img.resize(new_size, Image.LANCZOS)
            redim = True

        info = {
            'original': {'width': dim_orig[0], 'height': dim_orig[1]},
            'procesada': {'width': img.size[0], 'height': img.size[1], 'redimensionada': redim}
        }
        return img, info

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """Guarda JSON"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)

    @staticmethod
    def guardar_mascara_npz(mascara: np.ndarray, ruta: Path):
        """Guarda máscara en NPZ comprimido"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        np.savez_compressed(ruta, mascara=mascara.astype(np.float32))

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def obtener_memoria_gpu() -> Dict:
        """Info de memoria GPU"""
        if not torch.cuda.is_available():
            return {'disponible': False}
        return {
            'disponible': True,
            'nombre': torch.cuda.get_device_name(0),
            'asignada_mb': round(torch.cuda.memory_allocated(0) / 1024**2, 2),
            'reservada_mb': round(torch.cuda.memory_reserved(0) / 1024**2, 2)
        }

In [ ]:
# =============================================================================
# FILTRADO DE PERSONAS CON HEURÍSTICAS PARA RETRATO
# =============================================================================

class FiltradorPersonasRetrato:
    """
    Filtrado de detecciones para fotografía de retrato.

    Implementa heurísticas basadas en el estado del arte:
    - Lin et al. (2014) "Microsoft COCO: Common Objects in Context"
    - Dollar et al. (2012) "Pedestrian Detection: An Evaluation"
    - Best practices de fotografía de retrato

    Criterios de filtrado (mínimo 3 de 5):
    1. Área: 5%-90% de imagen (personas significativas)
    2. Aspect ratio: 0.3-1.5 (personas más altas que anchas)
    3. Confianza YOLO: >= umbral configurado
    4. Posición central: Distancia <= 0.35 (retratos suelen ser centrados)
    5. Compacidad: >0.15 (evitar detecciones fragmentadas)
    """

    def __init__(self):
        # Parámetros basados en análisis de datasets de retrato
        self.MIN_AREA_RATIO = 0.05      # Personas secundarias mínimas
        self.MAX_AREA_RATIO = 0.90      # Evitar detecciones de imagen completa
        self.MIN_ASPECT_RATIO = 0.3     # Personas muy anchas (sentadas/horizontales)
        self.MAX_ASPECT_RATIO = 1.5     # Personas muy altas (de pie/verticales)
        self.PREFERRED_CENTER_DISTANCE = 0.35  # Distancia normalizada al centro
        self.MIN_COMPACTNESS = 0.15     # Compacidad mínima (evitar fragmentación)

    def filtrar_detecciones(
        self,
        boxes: np.ndarray,
        masks: np.ndarray,
        scores: np.ndarray,
        imagen_shape: Tuple[int, int]
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, List[Dict]]:
        """
        Filtra detecciones que probablemente son personas en retratos.

        Args:
            boxes: Array (N, 4) con boxes en formato xyxy
            masks: Array (N, H, W) con máscaras binarias
            scores: Array (N,) con scores de confianza
            imagen_shape: (H, W) de la imagen

        Returns:
            Tuple con arrays filtrados y metadatos de cada detección
        """
        if len(boxes) == 0:
            return (np.array([]), np.array([]), np.array([]), [])

        h, w = imagen_shape
        area_total = h * w
        centro_x, centro_y = w / 2, h / 2

        indices_validos = []
        metadatos = []

        for idx in range(len(boxes)):
            box = boxes[idx]
            mascara = masks[idx]
            score = scores[idx]

            # Geometría del box
            x1, y1, x2, y2 = box
            box_w = x2 - x1
            box_h = y2 - y1
            box_area = box_w * box_h

            # Área de la máscara
            mask_area = np.sum(mascara > 0.5)
            area_ratio = mask_area / area_total

            # Aspect ratio
            aspect_ratio = box_w / box_h if box_h > 0 else 0

            # Distancia al centro
            box_centro_x = (x1 + x2) / 2
            box_centro_y = (y1 + y2) / 2
            dist_centro = np.sqrt(
                ((box_centro_x - centro_x) / w) ** 2 +
                ((box_centro_y - centro_y) / h) ** 2
            )

            # Compacidad (relación área máscara / área bbox)
            compacidad = mask_area / box_area if box_area > 0 else 0

            # Evaluar criterios
            criterios_cumplidos = 0

            if self.MIN_AREA_RATIO <= area_ratio <= self.MAX_AREA_RATIO:
                criterios_cumplidos += 1

            if self.MIN_ASPECT_RATIO <= aspect_ratio <= self.MAX_ASPECT_RATIO:
                criterios_cumplidos += 1

            # Confianza siempre cumplida (ya filtrada por YOLO)
            criterios_cumplidos += 1

            if dist_centro <= self.PREFERRED_CENTER_DISTANCE:
                criterios_cumplidos += 1

            if compacidad > self.MIN_COMPACTNESS:
                criterios_cumplidos += 1

            # Score de confianza de persona (0-1)
            person_confidence = criterios_cumplidos / 5.0

            # Aceptar si cumple al menos 3 de 5 criterios
            if criterios_cumplidos >= 3:
                indices_validos.append(idx)
                metadatos.append({
                    'id': idx,
                    'area_pixels': int(mask_area),
                    'area_ratio': float(area_ratio),
                    'aspect_ratio': float(aspect_ratio),
                    'center_distance': float(dist_centro),
                    'compactness': float(compacidad),
                    'criteria_met': criterios_cumplidos,
                    'person_confidence': person_confidence,
                    'yolo_confidence': float(score)
                })

        # Filtrar arrays
        if indices_validos:
            boxes_filtrados = boxes[indices_validos]
            masks_filtrados = masks[indices_validos]
            scores_filtrados = scores[indices_validos]

            # Ordenar por person_confidence descendente
            orden = np.argsort([m['person_confidence'] for m in metadatos])[::-1]
            boxes_filtrados = boxes_filtrados[orden]
            masks_filtrados = masks_filtrados[orden]
            scores_filtrados = scores_filtrados[orden]
            metadatos = [metadatos[i] for i in orden]

            return boxes_filtrados, masks_filtrados, scores_filtrados, metadatos
        else:
            return np.array([]), np.array([]), np.array([]), []


In [ ]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """Maneja checkpoint para retomar procesamiento"""

    def __init__(self, ruta: Path):
        self.ruta = ruta
        self.datos = self._cargar()

    def _cargar(self) -> Dict:
        if self.ruta.exists():
            with open(self.ruta, 'r') as f:
                return json.load(f)
        return {
            'completadas': [],
            'timestamp': datetime.now().isoformat()
        }

    def guardar(self):
        self.ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(self.ruta, 'w') as f:
            json.dump(self.datos, f, indent=2)

    def marcar_completada(self, nombre: str):
        if nombre not in self.datos['completadas']:
            self.datos['completadas'].append(nombre)
            self.guardar()

    def obtener_pendientes(self, todas: List[str]) -> List[str]:
        return [img for img in todas if img not in self.datos['completadas']]

In [ ]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES 3-PANEL
# =============================================================================

class GeneradorVisualizaciones:
    """
    Genera visualizaciones 3-panel avanzadas estilo SAM2.

    Panel 1: Imagen original sin modificar
    Panel 2: Bounding boxes + IDs + scores
    Panel 3: Máscaras coloreadas con overlay semi-transparente
    """

    @staticmethod
    def generar_visualizacion_3panel(
        imagen: np.ndarray,
        boxes: np.ndarray,
        mascaras: np.ndarray,
        scores: np.ndarray,
        metadatos: List[Dict],
        ruta_salida: Path,
        modelo_nombre: str,
        conf_threshold: float
    ):
        """
        Genera visualización 3-panel con información completa.

        Args:
            imagen: Array RGB (H, W, 3)
            boxes: Array (N, 4) con boxes xyxy
            mascaras: Array (N, H, W) con máscaras
            scores: Array (N,) con scores
            metadatos: Lista de diccionarios con métricas
            ruta_salida: Path para guardar PNG
            modelo_nombre: Nombre del modelo YOLO
            conf_threshold: Umbral de confianza usado
        """
        fig, axes = plt.subplots(1, 3, figsize=(24, 8))

        # PANEL 1: Original
        axes[0].imshow(imagen)
        axes[0].set_title('Imagen Original', fontsize=14, fontweight='bold')
        axes[0].axis('off')

        num_personas = len(boxes)

        if num_personas > 0:
            # Colores únicos para cada persona
            colors = plt.cm.tab20(np.linspace(0, 1, num_personas))

            # PANEL 2: Bounding boxes + info
            axes[1].imshow(imagen)

            for idx, (box, score, meta) in enumerate(zip(boxes, scores, metadatos)):
                x1, y1, x2, y2 = box
                color = colors[idx]

                # Bounding box
                rect = mpatches.Rectangle(
                    (x1, y1),
                    x2 - x1,
                    y2 - y1,
                    linewidth=3,
                    edgecolor=color,
                    facecolor='none'
                )
                axes[1].add_patch(rect)

                # Label con ID y score
                label_text = f"P{idx+1}\n{score:.2f}"
                axes[1].text(
                    x1, y1 - 10,
                    label_text,
                    color='white',
                    fontsize=11,
                    fontweight='bold',
                    bbox=dict(
                        boxstyle='round,pad=0.5',
                        facecolor=color,
                        alpha=0.9,
                        edgecolor='white',
                        linewidth=1
                    )
                )

            axes[1].set_title(
                f'Detecciones: {num_personas} persona(s) | conf≥{conf_threshold}',
                fontsize=14,
                fontweight='bold'
            )
            axes[1].axis('off')

            # PANEL 3: Máscaras coloreadas
            axes[2].imshow(imagen)

            for idx, (mascara, meta) in enumerate(zip(mascaras, metadatos)):
                color = colors[idx]

                # Overlay de máscara semi-transparente
                color_mask = np.zeros((*mascara.shape, 4))
                color_mask[mascara > 0.5] = [*color[:3], 0.5]  # Alpha 0.5
                axes[2].imshow(color_mask)

                # Añadir ID en el centro de la máscara
                coords = np.where(mascara > 0.5)
                if len(coords[0]) > 0:
                    center_y = int(np.mean(coords[0]))
                    center_x = int(np.mean(coords[1]))

                    axes[2].text(
                        center_x, center_y,
                        f"P{idx+1}",
                        color='white',
                        fontsize=16,
                        fontweight='bold',
                        ha='center',
                        va='center',
                        bbox=dict(
                            boxstyle='circle,pad=0.3',
                            facecolor=color,
                            alpha=0.9,
                            edgecolor='white',
                            linewidth=2
                        )
                    )

            axes[2].set_title(
                f'Máscaras de Segmentación',
                fontsize=14,
                fontweight='bold'
            )
            axes[2].axis('off')

        else:
            # Sin detecciones
            axes[1].imshow(imagen)
            axes[1].set_title(
                f'Sin detecciones | conf≥{conf_threshold}',
                fontsize=14,
                fontweight='bold'
            )
            axes[1].axis('off')

            axes[2].imshow(imagen)
            axes[2].set_title('Sin máscaras', fontsize=14, fontweight='bold')
            axes[2].axis('off')

        # Título general
        fig.suptitle(
            f'{modelo_nombre} - Portrait Segmentation',
            fontsize=16,
            fontweight='bold',
            y=0.98
        )

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

In [ ]:
# =============================================================================
# PROCESADOR YOLOV8
# =============================================================================

class ProcesadorYOLO:
    """Procesador optimizado para modelos YOLOv8-seg en fotografía de retrato"""

    def __init__(self, modelo_info: ModeloYOLO, config: ConfiguracionEvaluacion):
        self.modelo_info = modelo_info
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.filtrador = FiltradorPersonasRetrato()

        print(f"  Cargando: {modelo_info.nombre}")
        print(f"  Device: {self.device}")

        # Cargar modelo YOLO
        self.model = YOLO(modelo_info.archivo_pesos)
        self.model.to(self.device)

        print(f"  Modelo cargado correctamente")

        # ID de clase "person" en COCO (clase 0)
        self.person_class_id = 0

    def procesar_imagen(
        self,
        ruta_imagen: Path,
        config_umbral: ConfiguracionUmbrales,
        ruta_resultados: Path
    ) -> Dict:
        """
        Procesa una imagen con todos los umbrales de una configuración.
        Guarda UN JSON POR IMAGEN (no consolidado).

        Args:
            ruta_imagen: Path a la imagen
            config_umbral: ConfiguracionUmbrales con múltiples valores
            ruta_resultados: Directorio para guardar resultados

        Returns:
            Diccionario con resultados completos
        """

        nombre_foto = ruta_imagen.stem
        timestamp_inicio = time.time()

        # Cargar imagen
        img_pil, info_dim = Utilidades.cargar_imagen(ruta_imagen, self.config.max_dimension)
        img_array = np.array(img_pil)
        h_img, w_img = img_array.shape[:2]

        # Info GPU inicial
        gpu_ini = Utilidades.obtener_memoria_gpu()

        # Inferencia con umbral mínimo para obtener todas las detecciones
        t_inf_ini = time.time()

        results = self.model.predict(
            img_array,
            imgsz=self.config.img_size,
            conf=min(config_umbral.valores_conf),  # Umbral mínimo
            iou=config_umbral.iou_threshold,        # NMS threshold
            max_det=config_umbral.max_det,          # Máximo de detecciones
            classes=[self.person_class_id],         # Solo clase person
            device=self.device,
            verbose=False,
            retina_masks=True  # Máscaras de alta resolución
        )[0]

        tiempo_inferencia = (time.time() - t_inf_ini) * 1000

        # GPU pico
        gpu_pico = Utilidades.obtener_memoria_gpu()

        # Procesar por umbral
        resultados_umbrales = {}

        for umbral in config_umbral.valores_conf:
            resultado_umbral = self._procesar_umbral(
                results,
                img_pil,
                umbral,
                nombre_foto,
                ruta_resultados,
                (h_img, w_img)
            )
            resultados_umbrales[f'umbral_{umbral}'] = resultado_umbral

        # GPU final
        gpu_fin = Utilidades.obtener_memoria_gpu()

        tiempo_total = (time.time() - timestamp_inicio) * 1000

        # Construir resultado completo PARA ESTA IMAGEN
        resultado = {
            'timestamp': datetime.now().isoformat(),
            'nombre_imagen': ruta_imagen.name,
            'modelo': self.modelo_info.nombre_corto,
            'dataset': 'COCO',
            'configuracion': config_umbral.nombre,
            'arquitectura': self.modelo_info.nombre,
            'modelo_info': {
                'tamaño': self.modelo_info.tamaño,
                'parametros': self.modelo_info.parametros
            },
            'dimensiones': info_dim,
            'configuracion_inferencia': {
                'iou_threshold': config_umbral.iou_threshold,
                'max_det': config_umbral.max_det,
                'img_size': self.config.img_size,
                'umbrales_conf': config_umbral.valores_conf
            },
            'procesamiento': {
                'tiempo_inferencia_ms': round(tiempo_inferencia, 2),
                'tiempo_total_ms': round(tiempo_total, 2),
                'gpu_inicial_mb': gpu_ini.get('asignada_mb', 0),
                'gpu_pico_mb': gpu_pico.get('asignada_mb', 0),
                'gpu_final_mb': gpu_fin.get('asignada_mb', 0),
                'dispositivo': gpu_ini.get('nombre', 'CPU')
            },
            'resultados': resultados_umbrales
        }

        # GUARDAR JSON INDIVIDUAL POR IMAGEN
        ruta_json = ruta_resultados / "json" / f"{nombre_foto}.json"
        ruta_json.parent.mkdir(parents=True, exist_ok=True)
        Utilidades.guardar_json(resultado, ruta_json)

        return resultado

    def _procesar_umbral(
        self,
        results,
        img_pil: Image.Image,
        umbral: float,
        nombre_foto: str,
        ruta_resultados: Path,
        imagen_shape: Tuple[int, int]
    ) -> Dict:
        """
        Procesa un umbral específico con filtrado avanzado.

        Args:
            results: Resultados de YOLO
            img_pil: Imagen PIL
            umbral: Umbral de confianza actual
            nombre_foto: Nombre base de la foto
            ruta_resultados: Directorio de resultados
            imagen_shape: (H, W) de la imagen

        Returns:
            Diccionario con resultados del umbral
        """

        # Extraer detecciones que superan el umbral
        boxes_raw = []
        masks_raw = []
        scores_raw = []

        if results.masks is not None and len(results.masks) > 0:
            for idx in range(len(results.boxes)):
                box = results.boxes[idx]
                conf = float(box.conf[0])

                # Filtrar por umbral
                if conf >= umbral:
                    # Box en formato xyxy
                    bbox_xyxy = box.xyxy[0].cpu().numpy()
                    boxes_raw.append(bbox_xyxy)

                    # Máscara redimensionada
                    mascara = results.masks[idx].data[0].cpu().numpy()
                    h, w = img_pil.size[1], img_pil.size[0]
                    mascara_resize = cv2.resize(
                        mascara,
                        (w, h),
                        interpolation=cv2.INTER_LINEAR
                    )
                    masks_raw.append(mascara_resize)
                    scores_raw.append(conf)

        # Convertir a arrays numpy
        if boxes_raw:
            boxes_array = np.array(boxes_raw)
            masks_array = np.array(masks_raw)
            scores_array = np.array(scores_raw)

            # Aplicar filtrado inteligente para retrato
            boxes_filtradas, masks_filtradas, scores_filtradas, metadatos = \
                self.filtrador.filtrar_detecciones(
                    boxes_array,
                    masks_array,
                    scores_array,
                    imagen_shape
                )
        else:
            boxes_filtradas = np.array([])
            masks_filtradas = np.array([])
            scores_filtradas = np.array([])
            metadatos = []

        num_detecciones_raw = len(boxes_raw)
        num_personas = len(boxes_filtradas)

        print(f"    [Umbral {umbral}] Detecciones brutas: {num_detecciones_raw} | "
              f"Personas filtradas: {num_personas}")

        # Guardar máscaras si hay personas
        ruta_mascara_npz = None
        ruta_visualizacion = None

        if num_personas > 0:
            umbral_str = str(umbral).replace('.', '_')
            nombre_npz = f"{nombre_foto}_umbral{umbral_str}.npz"
            ruta_mascara_npz = ruta_resultados / "mascaras" / nombre_npz

            # NPZ simplificado: solo máscaras binarias
            # La imagen original está en 0_Imagenes
            # La geometría se calcula en notebook 03 desde las máscaras
            ruta_mascara_npz.parent.mkdir(parents=True, exist_ok=True)
            np.savez_compressed(
                ruta_mascara_npz,
                masks=masks_filtradas.astype(np.uint8)
            )

            # Generar visualización 3-panel
            nombre_vis = f"{nombre_foto}_umbral{umbral_str}.png"
            ruta_visualizacion = ruta_resultados / "visualizaciones" / nombre_vis

            GeneradorVisualizaciones.generar_visualizacion_3panel(
                np.array(img_pil),
                boxes_filtradas,
                masks_filtradas,
                scores_filtradas,
                metadatos,
                ruta_visualizacion,
                self.modelo_info.nombre,
                umbral
            )

        # Construir resultado del umbral
        resultado = {
            'umbral_confianza': umbral,
            'detecciones_brutas': num_detecciones_raw,
            'personas_filtradas': num_personas,
            'estadisticas_personas': metadatos,
            'ruta_mascaras_npz': str(ruta_mascara_npz) if ruta_mascara_npz else None,
            'ruta_visualizacion': str(ruta_visualizacion) if ruta_visualizacion else None,
            'scores_yolo': {
                'min': float(scores_filtradas.min()) if len(scores_filtradas) > 0 else None,
                'max': float(scores_filtradas.max()) if len(scores_filtradas) > 0 else None,
                'mean': float(scores_filtradas.mean()) if len(scores_filtradas) > 0 else None
            }
        }

        return resultado

In [ ]:
# =============================================================================
# EVALUADOR PRINCIPAL
# =============================================================================

class EvaluadorYOLO:
    """Coordinador principal del sistema de evaluación"""

    def __init__(self, config: ConfiguracionEvaluacion):
        self.config = config

        # Crear estructura de directorios
        self.dir_ejecucion = config.ruta_resultados / f"evaluacion"
        self.dir_ejecucion.mkdir(parents=True, exist_ok=True)

        print(f"\n{'='*80}")
        print(f"EVALUADOR YOLOv8-SEGMENTATION")
        print(f"{'='*80}")
        print(f"Directorio de ejecución: {self.dir_ejecucion}")
        print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

    def evaluar_modelo(
        self,
        modelo_key: str,
        config_umbral_key: str
    ) -> Optional[Path]:
        """Evalúa un modelo con una configuración de umbrales"""

        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        config_umbral = CONFIGURACIONES_UMBRALES[config_umbral_key]

        print(f"\n{'='*80}")
        print(f"MODELO: {modelo_info.nombre}")
        print(f"CONFIG: {config_umbral.descripcion}")
        print(f"UMBRALES: {config_umbral.valores_conf}")
        print(f"{'='*80}")

        # Crear directorios específicos (como SAM2)
        dir_modelo = self.dir_ejecucion / f"{modelo_info.nombre_corto}_{config_umbral.nombre}"
        dir_modelo.mkdir(parents=True, exist_ok=True)
        (dir_modelo / "mascaras").mkdir(exist_ok=True)
        (dir_modelo / "visualizaciones").mkdir(exist_ok=True)
        (dir_modelo / "json").mkdir(exist_ok=True)  # JSON por imagen

        # Checkpoint
        checkpoint_path = dir_modelo / "checkpoint.json"
        checkpoint = GestorCheckpoint(checkpoint_path)

        # Obtener imágenes
        imagenes = sorted(self.config.ruta_imagenes.glob("*.jpg"))
        if not imagenes:
            imagenes = sorted(self.config.ruta_imagenes.glob("*.png"))

        pendientes = checkpoint.obtener_pendientes([img.name for img in imagenes])
        imagenes_procesar = [img for img in imagenes if img.name in pendientes]

        print(f"\nImágenes totales: {len(imagenes)}")
        print(f"Completadas: {len(imagenes) - len(pendientes)}")
        print(f"Pendientes: {len(pendientes)}")

        if not imagenes_procesar:
            print("OK Todas las imágenes ya procesadas")
            return dir_modelo

        # Cargar modelo
        try:
            procesador = ProcesadorYOLO(modelo_info, self.config)
        except Exception as e:
            print(f"ERROR Error cargando modelo: {e}")
            return None

        # Procesar imágenes (cada una guarda su propio JSON)
        for idx, imagen_path in enumerate(imagenes_procesar, 1):
            print(f"\n[{idx}/{len(imagenes_procesar)}] {imagen_path.name}")

            try:
                # Procesar imagen (guarda JSON automáticamente)
                procesador.procesar_imagen(
                    imagen_path,
                    config_umbral,
                    dir_modelo
                )

                # Marcar completada
                checkpoint.marcar_completada(imagen_path.name)

                # Pausa
                if idx < len(imagenes_procesar):
                    time.sleep(self.config.pausa_entre_imagenes)

                # Liberar memoria cada 10 imágenes
                if idx % 10 == 0:
                    Utilidades.liberar_memoria()

            except Exception as e:
                print(f"ERROR Error procesando {imagen_path.name}: {e}")
                import traceback
                traceback.print_exc()

        print(f"\nOK Configuración completada")
        return dir_modelo

    def ejecutar_evaluacion_completa(self) -> Dict[str, List[Path]]:
        """Ejecuta evaluación de todos los modelos configurados"""

        resultados_generados = {}

        total = len(self.config.modelos_evaluar) * len(self.config.configs_umbrales)
        actual = 0

        for modelo_key in self.config.modelos_evaluar:
            if modelo_key not in MODELOS_DISPONIBLES:
                print(f"ERROR Modelo '{modelo_key}' no encontrado")
                continue

            directorios_modelo = []

            for config_key in self.config.configs_umbrales:
                if config_key not in CONFIGURACIONES_UMBRALES:
                    print(f"ERROR Configuración '{config_key}' no encontrada")
                    continue

                actual += 1
                print(f"\n{'='*80}")
                print(f"PROGRESO: {actual}/{total}")
                print(f"{'='*80}")

                directorio = self.evaluar_modelo(modelo_key, config_key)

                if directorio:
                    directorios_modelo.append(directorio)

                # Pausa entre configuraciones
                if actual < total:
                    time.sleep(self.config.pausa_entre_modelos)

                # Liberar memoria
                Utilidades.liberar_memoria()

            if directorios_modelo:
                resultados_generados[modelo_key] = directorios_modelo

        return resultados_generados

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main():

    # Configuración optimizada para fotografía de retrato
    config = ConfiguracionEvaluacion(
        # TODOS los modelos YOLOv8-seg disponibles
        modelos_evaluar=['nano', 'small', 'medium', 'large', 'xlarge'],

        # TODAS las configuraciones de retrato
        configs_umbrales=[
            'fast_portrait',
            'balanced_portrait',
            'sensitive_portrait',
            'quality_portrait'
        ]
    )

    # Validar rutas
    if not config.ruta_imagenes.exists():
        print(f"ERROR: Directorio de imágenes no existe: {config.ruta_imagenes}")
        return

    # Crear evaluador
    evaluador = EvaluadorYOLO(config)

    # Ejecutar
    print("\nIniciando evaluación completa...")

    resultados = evaluador.ejecutar_evaluacion_completa()

    # Resumen final
    print(f"\n{'='*80}")
    print(f"EVALUACIÓN COMPLETADA")
    print(f"{'='*80}")
    print(f"Directorio: {evaluador.dir_ejecucion}")
    print(f"\nModelos evaluados: {len(resultados)}")
    for modelo, paths in resultados.items():
        print(f"  - {modelo}: {len(paths)} configuraciones")

    print(f"\nProceso finalizado exitosamente")

In [ ]:
if __name__ == "__main__":
    main()


EVALUADOR YOLOv8-SEGMENTATION
Directorio de ejecución: /content/drive/MyDrive/TFM/2_Modelos/yolov8/evaluacion
Device: GPU

Iniciando evaluación completa...

PROGRESO: 1/20

MODELO: YOLOv8 Nano Segmentation
CONFIG: Rápida - Detecta personas principales con alta confianza
UMBRALES: [0.25, 0.4, 0.5]

Imágenes totales: 20
Completadas: 20
Pendientes: 0
OK Todas las imágenes ya procesadas

PROGRESO: 2/20

MODELO: YOLOv8 Nano Segmentation
CONFIG: Equilibrada - Balance entre velocidad y detección de personas secundarias
UMBRALES: [0.15, 0.25, 0.35, 0.5]

Imágenes totales: 20
Completadas: 20
Pendientes: 0
OK Todas las imágenes ya procesadas

PROGRESO: 3/20

MODELO: YOLOv8 Nano Segmentation
CONFIG: Sensible - Detecta incluso personas parcialmente visibles o en segundo plano
UMBRALES: [0.05, 0.1, 0.2, 0.3, 0.4]

Imágenes totales: 20
Completadas: 20
Pendientes: 0
OK Todas las imágenes ya procesadas

PROGRESO: 4/20

MODELO: YOLOv8 Nano Segmentation
CONFIG: Calidad - Rango amplio para análisis de s